<a href="https://colab.research.google.com/github/ma-prietoo/SyS-2025-2/blob/main/Parcial%202/Parcial2_Sys_2025_2_MariaPrieto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Parcial 2




In [63]:
# Instalación de librerías
!pip install streamlit soundfile yt-dlp numpy matplotlib -q
!apt-get install ffmpeg -y -qq

# Instalar Cloudflare Tunnel
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -q
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

# Crear carpeta pages para trabajar Multiapp en Streamlit
!rm -rf pages
!mkdir pages


In [64]:
# Página principal
%%writefile app.py
import streamlit as st

st.set_page_config(page_title="Parcial 2")
st.title("Parcial 2 Señales y Sistemas")

st.markdown("""
Estudiante: Maria Prieto
""")
st.markdown("""
Documento: 1062278202""")
st.markdown("""
Universidad Nacional de Colombia
""")
st.markdown("""
2025-2""")


Overwriting app.py


In [65]:
%%writefile pages/1_Modulación_AM.py
import streamlit as st
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
import os
import shlex
import subprocess

st.title("1. Modulación AM")
st.markdown("""
Ejercicio:
""")
st.markdown("""
Sea el demodulador en amplitud presentado en la figura.
Asumiendo θ0 = 0, determine el espectro de Fourier (teorico) en cada una de las etapas del sistema. Luego, con base en la simulacion de modulación en amplitud del Taller 2 y utilizando cinco segundos de una cancion de Youtube como mensaje, grafique cada una de las etapas principales del proceso de modulacion y demodulación en el tiempo y la frecuencia (reproduzca el segmento de la cancion en cada etapa).
Nota: Para la etapa de filtrado pasa bajas, realice su implementacion a partir de la transformada rápida de Fourier.
""")

# Teoría
st.header("Teoría:")
st.markdown("""
La modulación AM consiste en multiplicar una señal de mensaje por una portadora sinusoidal.

Ecuación:
""")
st.latex(r"y(t) = (1 + m(t)) A_c \cos(2\pi f_c t)")

st.markdown("""
La demodulación coherente consiste en:
1. Multiplicar por la portadora local
2. Aplicar un filtro pasa bajas
3. Normalizar
""")

# Link de YouTube
st.header("Descargar Audio desde YouTube")

default_link = "https://www.youtube.com/watch?v=7PGKdzr26Hw"
link = st.text_input("Cambia el enlace de youtube:", value=default_link) # Enlace por defecto

if st.button("Descargar Audio"):

    with st.spinner("Descargando audio…"):
        # Eliminar archivos previos (si existen)
        for fn in ("audio.mp3", "audio.m4a", "audio.webm", "output.wav", "audio"):
            try:
                os.remove(fn)
            except Exception:
                pass

        # Construir comando de descarga con salida fija (extensible)
        # Se fuerza nombre audio.<ext> para luego convertir a wav
        cmd = f'yt-dlp --extract-audio -o "audio.%(ext)s" --audio-format mp3 --force-overwrites "{link}"'
        try:
            subprocess.run(shlex.split(cmd), check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        except subprocess.CalledProcessError as e:
            st.error("Error al descargar el audio con yt-dlp. Revisa el enlace o la instalación de yt-dlp.")
            st.code(e.stderr.decode("utf-8")[:1000])
            st.stop()

        # Detectar archivo descargado (buscar audio)
        downloaded = None
        for ext in ("mp3", "m4a", "webm", "wav"):
            candidate = f"audio.{ext}"
            if os.path.exists(candidate):
                downloaded = candidate
                break

        if downloaded is None:
            st.error("No se encontró el archivo descargado.")
            st.stop()

        # Convertir a WAV con ffmpeg (si no es wav)
        if not downloaded.lower().endswith(".wav"):
            cmd_conv = f'ffmpeg -y -i "{downloaded}" output.wav'
            try:
                subprocess.run(shlex.split(cmd_conv), check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                wav_path = "output.wav"
            except subprocess.CalledProcessError as e:
                st.error("Error al convertir a WAV con ffmpeg.")
                st.code(e.stderr.decode("utf-8")[:1000])
                st.stop()
        else:
            wav_path = downloaded

    st.success("Audio descargado y convertido")

    # Leer y extraer fragmento 20–25 s
    try:
        audio, fs = sf.read(wav_path)
    except Exception as e:
        st.error(f"No se pudo leer el archivo WAV: {e}")
        st.stop()

    if audio.ndim > 1:  # Si es estéreo, convertir a mono
        audio = np.mean(audio, axis=1)

    ti, tf = 20, 25 # Segmento de 20 a 25 segundos
    start = int(ti * fs)
    end = int(tf * fs)
    if end > len(audio):
        st.warning("El archivo es más corto que 25 s; se usará el tramo disponible al final.")
        start = max(0, len(audio) - int(5 * fs))
        end = len(audio)

    segmento = audio[start:end]
    if np.max(np.abs(segmento)) == 0:
        st.error("El segmento seleccionado es todo ceros.")
        st.stop()

    mensaje = segmento / np.max(np.abs(segmento)) # Normalización
    t_audio = np.arange(len(mensaje)) / fs # Tiempo para el gráfico

    # Reproducir el fragmento de audio
    st.subheader("Fragmento (20–25 s)")
    st.audio(mensaje, sample_rate=fs)

    # Modulación AM
    st.header("Modulación AM")

    # Parámetros para la modulación AM
    Fc = 5000  # Frecuencia de la portadora (5000 Hz)
    Ac = 1     # Amplitud de la portadora
    Im = 1     # Índice de modulación

    # Señal portadora c(t)
    portadora = Ac * np.cos(2 * np.pi * Fc * t_audio)
    # Señal modulada en AM: y(t) = (1 + m(t)) * Ac * cos(2πFc t)
    # Nota: se mantiene Im para ajustar índice si se desea
    y = (1 + Im * mensaje) * portadora

    # Normalizar y para evitar clipping al reproducir
    y = y / (np.max(np.abs(y)) + 1e-12)

    # Reproducir las señales
    st.audio(y, sample_rate=fs)

    # Función para calcular espectro de frecuencia usando FFT
    def espectro(x, Fs, N_fft=2**15):
        """
        Devuelve frecuencias y magnitud normalizada del espectro (valores positivos).
        """
        # Ventana para reducir fugas
        win = np.hanning(len(x))
        xw = x * win
        # Zero-pad o recortar a N_fft
        X = np.fft.rfft(xw, n=N_fft)
        f = np.fft.rfftfreq(N_fft, 1 / Fs)
        # Convertir a escala de magnitud adecuada (2/N para conservar energía en rfft)
        mag = (2.0 / np.sum(win)) * np.abs(X)
        return f, mag

    # Calcular espectros
    f_m, M = espectro(mensaje, fs)
    f_y, Y = espectro(y, fs)

    # Graficar señales en el tiempo
    st.subheader("Señales en el Tiempo")
    fig, ax = plt.subplots(4, 1, figsize=(10, 9), sharex=True)
    ax[0].plot(t_audio, mensaje); ax[0].set_title("Mensaje"); ax[0].grid()
    ax[1].plot(t_audio, portadora, color='orange'); ax[1].set_title("Portadora"); ax[1].grid()
    ax[2].plot(t_audio, y, color='green'); ax[2].set_title("Señal AM"); ax[2].grid()

    # Etapa de Mixer DSB-CS: Mezcla del mensaje con la portadora (mensaje * cos(2πFc t))
    # (mostramos la señal al mezclar para visualizar contenid o de banda lateral)
    mixer = mensaje * np.cos(2 * np.pi * Fc * t_audio)
    ax[3].plot(t_audio, mixer, color='purple'); ax[3].set_title("Señal después del Mixer DSB-CS"); ax[3].grid()

    # Ajustar el espacio entre subgráficos (hspace controla el espacio vertical)
    plt.subplots_adjust(hspace=0.6)

    st.pyplot(fig)

    # Graficar espectros de frecuencia
    st.subheader("Espectros de Frecuencia")
    fig2, ax2 = plt.subplots(3, 1, figsize=(10, 10))

    # Espectro del mensaje
    ax2[0].plot(f_m, M, label="Mensaje")
    ax2[0].set_xlim(0, min(12000, fs/2))
    ax2[0].set_title("Espectro del Mensaje")
    ax2[0].grid()

    # Espectro de la señal AM
    ax2[1].plot(f_y, Y, label="Señal AM")
    ax2[1].set_xlim(0, min(12000, fs/2))
    ax2[1].set_title("Espectro de la Señal AM")
    ax2[1].grid()

    # Espectro después del Mixer DSB-CS
    f_mixer, M_mixer = espectro(mixer, fs)
    ax2[2].plot(f_mixer, M_mixer, label="Después del Mixer DSB-CS")
    ax2[2].set_xlim(0, min(12000, fs/2))
    ax2[2].set_title("Espectro después del Mixer DSB-CS")
    ax2[2].grid()

    plt.subplots_adjust(hspace=0.6)

    st.pyplot(fig2)

    # Demodulación
    st.header("Demodulación")

    # Multiplicación de la señal modulada por la portadora local (coherente)
    demodulada = y * portadora

    # FFT de la señal demodulada
    N_dem = len(demodulada)
    X = np.fft.rfft(demodulada)
    freqs = np.fft.rfftfreq(N_dem, 1 / fs)

    # Filtro pasa-bajas
    cutoff = min(8000, fs / 2 - 1)
    X_f = X.copy()
    X_f[freqs > cutoff] = 0  # Frecuencia de corte del LPF

    # Reconstruir señal en tiempo
    mensaje_rec = np.fft.irfft(X_f, n=N_dem)

    # Compensación por la multiplicación por la portadora (factor 1/2) y por ventana si aplica
    # (en nuestro caso, hacemos una normalización sencilla)
    if np.max(np.abs(mensaje_rec)) > 0:
        mensaje_rec = mensaje_rec / np.max(np.abs(mensaje_rec))
    else:
        st.warning("Señal recuperada es prácticamente cero después del filtrado.")

    # Recortar/pad si por alguna razón la longitud difiere (asegurar misma longitud para graficar)
    L = min(len(mensaje), len(mensaje_rec))
    mensaje_rec = mensaje_rec[:L]
    mensaje_plot = mensaje[:L]
    t_plot = t_audio[:L]

    # Reproducir mensaje recuperado
    st.audio(mensaje_rec, sample_rate=fs)

    # Graficar mensaje original vs recuperado
    fig3, ax3 = plt.subplots(figsize=(10, 4))
    ax3.plot(t_plot, mensaje_plot, '--', label="Original")
    ax3.plot(t_plot, mensaje_rec, color='red', label="Recuperado")
    ax3.set_title("Mensaje original vs Recuperado")
    ax3.grid()
    ax3.legend()
    st.pyplot(fig3)

    st.success("Demodulación completada.")


Writing pages/1_Modulación_AM.py


In [66]:
%%writefile pages/2_Análisis_Masa-Resorte-Amortiguador.py

import streamlit as st
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

# Usado para poder poner formato latex a las ecuaciones
st.markdown("""
<script>
window.MathJax = {
  tex: {inlineMath: [['$', '$'], ['\\\\(', '\\\\)']]},
  svg: {fontCache: 'global'}
};
</script>
<script src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-svg.js"></script>
""", unsafe_allow_html=True)

st.set_page_config(page_title="Análisis Masa-Resorte-Amortiguador")

st.title("2. Análisis del Sistema Masa-Resorte-Amortiguador")

st.markdown("""
Ejercicio:
""")

st.markdown("""
Encuentre la funcion de transferencia que caracteriza el sistema masa, resorte, amortiguador, presentado en la figura (asuma condiciones iniciales cero).
Posteriormente, encuentre el sistema equivalente del modelo masa, resorte, amortiguador, a partir del circuito eléctrico.
Finalmente, proponga unos valores de m, k y c y sus equivalentes R, L y C, para simular un sistema subamortiguado, sobreamortiguado y de amortiguamiento crítico (determine el factor de amortiguamiento, la frecuencia natural amortiguada, la frecuencia natural no amortiguada, el tiempo pico, tiempo de levantamiento y el tiempo de establecimiento en cada caso). Para cada caso, grafique el diagrama de polos y ceros, el diagrama de Bode, la respuesta impulso, respuesta escalón y respuesta rampa. Repita el proceso para modo lazo cerrado.

""")

st.markdown(r"""
### Ecuación diferencial del sistema

$$
m \frac{d^2 y(t)}{dt^2} + c \frac{dy(t)}{dt} + k\,y(t) = F_E(t)
$$
""")

st.markdown("""
Donde:
- \( m \): masa
- \( c \): coeficiente de amortiguamiento
- \( k \): constante del resorte
- \( F_E(t) \): fuerza externa
- \( y(t) \): desplazamiento
""")

st.markdown("""
### Fórmulas importantes:
""")

st.markdown(r"""
Frecuencia natural no amortiguada:
$$
\omega_n = \sqrt{\frac{k}{m}}
$$

Frecuencia amortiguada:
$$
\omega_d = \omega_n \sqrt{1 - \xi^2}
$$

Tiempo pico:
$$
t_p = \frac{\pi}{\omega_d}
$$

Tiempo de levantamiento:
$$
t_u = \frac{3}{\xi\,\omega_n}
$$

Tiempo de establecimiento:
$$
t_s = \frac{4}{\xi\,\omega_n}
$$
""")

# Parámetros ajustables
st.sidebar.header("Parámetros del Sistema")

m = st.sidebar.number_input("Masa (m)", min_value=0.01, value=0.1, step=0.01)
k = st.sidebar.number_input("Constante del Resorte (k)", min_value=0.01, value=2.5, step=0.1)
c = st.sidebar.number_input("Coeficiente de Amortiguamiento (c)", min_value=0.01, value=0.1, step=0.01)
xi = st.sidebar.slider("Factor de Amortiguamiento (ξ)", 0.01, 2.0, 0.5, 0.01)

lazo = st.sidebar.radio("Tipo de sistema", ("Lazo Abierto", "Lazo Cerrado"))

# Cálculos de frecuencias y tiempos


omega_n = np.sqrt(k / m)
omega_d = omega_n * np.sqrt(np.abs(1 - xi**2))
t_p = np.pi / omega_d
t_u = 3 / (xi * omega_n)
t_s = 4 / (xi * omega_n)

st.markdown(
    fr"""
### Resultados Numéricos

**Frecuencia natural no amortiguada**
$$
\omega_n = {omega_n:.2f}\ \text{{rad/s}}
$$

**Frecuencia amortiguada**
$$
\omega_d = {omega_d:.2f}\ \text{{rad/s}}
$$

**Tiempo pico**
$$
t_p = {t_p:.2f}\ \text{{s}}
$$

**Tiempo de levantamiento**
$$
t_u = {t_u:.2f}\ \text{{s}}
$$

**Tiempo de establecimiento**
$$
t_s = {t_s:.2f}\ \text{{s}}
$$
"""
)

# Función de transferencia
num = [1]
den = [m, c, k]

if lazo == "Lazo Cerrado":
    den = np.polyadd([1], den)

sys = signal.TransferFunction(num, den)

# Diagrama de Bode

st.subheader("Diagrama de Bode")

w, mag, phase = signal.bode(sys)
fig_bode, ax = plt.subplots(2, 1, figsize=(8, 6))

ax[0].semilogx(w, mag)
ax[0].set_title("Magnitud")
ax[0].set_ylabel("Magnitud [dB]")

ax[1].semilogx(w, phase)
ax[1].set_title("Fase")
ax[1].set_ylabel("Fase [°]")
ax[1].set_xlabel("Frecuencia [rad/s]")

st.pyplot(fig_bode)

# Diagrama de polos y ceros
st.subheader("Diagrama de Polos y Ceros")

zeros, poles, _ = signal.tf2zpk(sys.num, sys.den)

fig_pz, ax = plt.subplots(figsize=(8, 6))
ax.scatter(np.real(zeros), np.imag(zeros), c='red', marker='o', label="Ceros")
ax.scatter(np.real(poles), np.imag(poles), c='blue', marker='x', label="Polos")

ax.axhline(0, color='black')
ax.axvline(0, color='black')

ax.set_xlabel("Parte Real")
ax.set_ylabel("Parte Imaginaria")
ax.legend()
ax.set_title("Polos y Ceros")

st.pyplot(fig_pz)

# Respuestas del sistema

st.subheader("Respuestas del Sistema")

t = np.linspace(0, 10, 1000)

t_imp, y_imp = signal.impulse(sys, T=t)
t_step, y_step = signal.step(sys, T=t)

rampa = t
t_ramp, y_ramp, _ = signal.lsim(sys, rampa, t)

fig_resp, ax = plt.subplots(3, 1, figsize=(8, 6))

ax[0].plot(t_imp, y_imp)
ax[0].set_title("Respuesta al Impulso")

ax[1].plot(t_step, y_step)
ax[1].set_title("Respuesta al Escalón")

ax[2].plot(t_ramp, y_ramp)
ax[2].set_title("Respuesta a la Rampa")
ax[2].set_xlabel("Tiempo [s]")

plt.tight_layout()
st.pyplot(fig_resp)


Writing pages/2_Análisis_Masa-Resorte-Amortiguador.py


In [67]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run app.py &>/content/logs.txt &

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

--2025-12-03 05:39:45--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2025-12-03 05:39:45--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-03T06%3A21%3A03Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-03

In [68]:
#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        if "Your quick Tunnel has been created" in line:
            found_context = True

        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  # Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break


Tu aplicación está disponible en: https://bits-vermont-wines-mall.trycloudflare.com


In [69]:
import os

res = input("Digite (0) para finalizar la ejecución del Dashboard: ")

if res.upper() == "0":
    os.system("pkill streamlit")  # Termina el proceso de Streamlit
    print("La aplicación se ha cerrado.")


Digite (0) para finalizar la ejecución del Dashboard: 0
La aplicación se ha cerrado.
